In [1]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# pip install tensorflow

In [3]:


# 標準使用ライブラリー
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc
import os
import shutil


# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 追記
import json
import datetime
import math
plt.style.use('dark_background')

# debug
#%pdb on

import pixiedust #%pixie_debugger
import os
os.environ["CUDA_VISIBLE_DEVICES"]="5"

# tfがエラーはかないため
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


Pixiedust database opened successfully


In [4]:
# 画像サイズ
img_size = 128
channels = 3
img_shape = (img_size, img_size, channels)

# 学習の設定
batch_size = 32
epochs = 100

画像のデータジェネレータ

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def preprocessing_function(x):
    return x / 127.5 - 1

A_datagen = ImageDataGenerator(width_shift_range=0.1,
                 height_shift_range=0.1,
                 preprocessing_function = preprocessing_function
                 )
B_datagen = ImageDataGenerator(width_shift_range=0.1,
                 height_shift_range=0.1,
                 preprocessing_function = preprocessing_function
                 )

# apple画像へのpathを渡す
A_generator = A_datagen.flow_from_directory('./apple2orange/trainA',
                                            class_mode=None,
                                            batch_size=batch_size,
                                            target_size=(img_size, img_size), 
                                            color_mode='rgb')

# orange画像へのpathを渡す
B_generator = B_datagen.flow_from_directory('./apple2orange/trainB',
                                            class_mode=None, 
                                            batch_size=batch_size,
                                            target_size=(img_size, img_size), 
                                            color_mode='rgb')

Found 995 images belonging to 1 classes.
Found 1019 images belonging to 1 classes.


モデルの作成

In [6]:
from tensorflow.keras.initializers import RandomNormal

kernel_init = RandomNormal(mean=0.0, stddev=0.02)
gamma_init = RandomNormal(mean=0.0, stddev=0.02)

In [7]:
# ! pip install tensorflow_addons 

In [8]:
import tensorflow.keras.backend as K

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, UpSampling2D, BatchNormalization, Conv2D, Activation, Dropout, Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU, LayerNormalization, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

from tensorflow_addons.layers import InstanceNormalization


patch_size = int(img_size/2**4)
patch_shape = (patch_size, patch_size, 1)


def build_generator(filters=32):
    
    model_input = Input(shape=img_shape)
    
    # Encoder
    x = Conv2D(filters, kernel_size=4, strides=2, padding="same", use_bias=False, kernel_initializer=kernel_init)(model_input)
    x = LeakyReLU(0.2)(x)
    x1 = InstanceNormalization(gamma_initializer=gamma_init)(x)
    
    x = Conv2D(filters*2, kernel_size=4, strides=2, padding="same", use_bias=False, kernel_initializer=kernel_init)(x1)
    x = LeakyReLU(0.2)(x)
    x2 = InstanceNormalization(gamma_initializer=gamma_init)(x)
    
    x = Conv2D(filters*4, kernel_size=4, strides=2, padding="same", use_bias=False, kernel_initializer=kernel_init)(x2)
    x = LeakyReLU(0.2)(x)
    x3 = InstanceNormalization()(x)  
    
    x = Conv2D(filters*8, kernel_size=4, strides=2, padding="same", use_bias=False, kernel_initializer=kernel_init)(x3)
    x = LeakyReLU(0.2)(x)
    x4 = InstanceNormalization(gamma_initializer=gamma_init)(x)

    
    # Decoder
    x = UpSampling2D((2, 2))(x4)
    x = Conv2D(filters*4, kernel_size=4, strides=1, padding="same", use_bias=False, kernel_initializer=kernel_init)(x)
    x = LeakyReLU(0.2)(x)
    x = InstanceNormalization(gamma_initializer=gamma_init)(x)  
    x = Concatenate()([x, x3])
    
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(filters*2, kernel_size=4, strides=1, padding="same", use_bias=False, kernel_initializer=kernel_init)(x)
    x = LeakyReLU(0.2)(x)
    x = InstanceNormalization(gamma_initializer=gamma_init)(x)  
    x = Concatenate()([x, x2])
    
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(filters, kernel_size=4, strides=1, padding="same", use_bias=False, kernel_initializer=kernel_init)(x)
    x = LeakyReLU(0.2)(x)
    x = InstanceNormalization(gamma_initializer=gamma_init)(x)  
    x = Concatenate()([x, x1])
    
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(3, kernel_size=4, strides=1, padding="same", activation="tanh", use_bias=False, kernel_initializer=kernel_init)(x)

    model = Model(model_input, x)
    
    model.summary()
    
    return model


def build_discriminator(filters=64):
    
    model_input = Input(shape=img_shape)
    
    x = Conv2D(filters, kernel_size=4, strides=2, padding="same", use_bias=False, kernel_initializer=kernel_init)(model_input)
    x = LeakyReLU(0.2)(x)
    
    x = Conv2D(filters*2, kernel_size=4, strides=2, padding="same", use_bias=False, kernel_initializer=kernel_init)(x)
    x = LeakyReLU(0.2)(x)
    x = InstanceNormalization(gamma_initializer=gamma_init)(x)
    
    x = Conv2D(filters*4, kernel_size=4, strides=2, padding="same", use_bias=False, kernel_initializer=kernel_init)(x)
    x = LeakyReLU(0.2)(x)
    x = InstanceNormalization(gamma_initializer=gamma_init)(x)  
    
    x = Conv2D(filters*8, kernel_size=4, strides=2, padding="same", use_bias=False, kernel_initializer=kernel_init)(x)
    x = LeakyReLU(0.2)(x)
    x = InstanceNormalization(gamma_initializer=gamma_init)(x)
    
    x = Conv2D(1, kernel_size=4, strides=1, padding="same", use_bias=False, kernel_initializer=kernel_init)(x)
    
    model = Model(model_input, x)
    
    model.summary()
    
    return model

In [9]:
from tensorflow.keras import optimizers

optimizer = optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

lambda_cycyle = 10.
lambda_id = 5.

D_A = build_discriminator()
D_B = build_discriminator()

D_A.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
D_B.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

G_AB = build_generator()
G_BA = build_generator()

# 入力画像のためのプレースホルダー
real_A = Input(shape=img_shape)
real_B = Input(shape=img_shape)

# 生成器一回でできるフェイク画像
fake_A = G_BA(real_B)
fake_B = G_AB(real_A)

# サイクル整合性のための再構成
cycle_A = G_BA(fake_B)
cycle_B = G_AB(fake_A)

# 同一性
same_A = G_BA(real_A)
same_B = G_AB(real_B)

# fake画像に対する識別器の出力
disc_fake_A = D_A(fake_A)
disc_fake_B = D_B(fake_B)

D_A.trainable = False
D_B.trainable = False

cycle_gan = Model(inputs=[real_A, real_B],
                 outputs=[disc_fake_A, disc_fake_B,
                         cycle_A, cycle_B,
                         same_A, same_B])

cycle_gan.compile(loss=['mse', 'mse', 'mae', 'mae', 'mae', 'mae'],
                 loss_weights=[1, 1, lambda_cycyle, lambda_cycyle, lambda_id, lambda_id],
                 optimizer=optimizer)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 64)        3072      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       131072    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 128)       0         
_________________________________________________________________
instance_normalization (Inst (None, 32, 32, 128)       256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)      

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 64, 64, 32)   1536        input_4[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_15 (LeakyReLU)      (None, 64, 64, 32)   0           conv2d_18[0][0]                  
__________________________________________________________________________________________________
instance_normalization_13 (Inst (None, 64, 64, 32)   64          leaky_re_lu_15[0][0]             
_______________________________________________________________________________________

学習

In [10]:
disc_A_loss_hoistory = []
disc_B_loss_hoistory = []
disc_loss_hoistory = []

gen_loss_hoistory = []


def train(epochs, iterations_per_epochs, batch_size=128):
    
    valid = np.ones((batch_size,) + patch_shape)
    fake = np.zeros((batch_size,) + patch_shape)
    
    for epoch in range(epochs):
        for i in range(iterations_per_epochs):
            
            batch_A = next(A_generator)
            batch_B = next(B_generator)
            
            # generatorのミニバッチの余りの処理。もっと実装を賢くし直した方がいい
            batch_size_modified = batch_size
            if batch_A.shape[0]!=batch_B.shape[0]:
                batch_size_modified = min(batch_A.shape[0], batch_B.shape[0])
                
            batch_A = batch_A[:batch_size_modified]
            batch_B = batch_B[:batch_size_modified]
            
            fake_B = G_AB.predict(batch_A)
            fake_A = G_BA.predict(batch_B)
            
            disc_A_loss_real = D_A.train_on_batch(batch_A, valid[:batch_size_modified])
            disc_A_loss_fake = D_A.train_on_batch(fake_A, fake[:batch_size_modified])
            
            disc_A_loss = 0.5 * np.add(disc_A_loss_real, disc_A_loss_fake)
            disc_A_loss_hoistory.append(disc_A_loss)
            
            disc_B_loss_real = D_B.train_on_batch(batch_B, valid[:batch_size_modified])
            disc_B_loss_fake = D_B.train_on_batch(fake_B, fake[:batch_size_modified])
            
            disc_B_loss = 0.5 * np.add(disc_B_loss_real, disc_B_loss_fake)
            disc_B_loss_hoistory.append(disc_B_loss)
            
            d_loss = 0.5 * np.add(disc_A_loss, disc_B_loss)
            disc_loss_hoistory.append(d_loss)
            
            g_loss = cycle_gan.train_on_batch([batch_A, batch_B],
                                              [valid[:batch_size_modified], valid[:batch_size_modified],
                                               batch_A, batch_B,
                                               batch_A, batch_B])
            gen_loss_hoistory.append(g_loss)
            
            if i==0:
                print('d loss, g loss:', d_loss[0],  g_loss)

In [11]:
iterations_per_epochs = 995//batch_size

train(epochs=epochs, iterations_per_epochs=iterations_per_epochs, batch_size=batch_size)

d loss, g loss: 0.4999896071653893 [19.736116409301758, 0.9851574897766113, 0.986069917678833, 0.6219570636749268, 0.5619264841079712, 0.6241270303726196, 0.5610835552215576]
d loss, g loss: 0.1297882068902254 [13.115886688232422, 0.5773209929466248, 0.6009085178375244, 0.43464386463165283, 0.3706207275390625, 0.4226565361022949, 0.35434582829475403]
d loss, g loss: 0.07287528645247221 [11.63404655456543, 0.9550924301147461, 0.7979224920272827, 0.31753867864608765, 0.3552025556564331, 0.2955474257469177, 0.3351762592792511]
d loss, g loss: 0.11887113749980927 [10.5689697265625, 0.9242114424705505, 0.8406267762184143, 0.2928023338317871, 0.3047705292701721, 0.28133970499038696, 0.2843409776687622]
d loss, g loss: 0.16990651562809944 [9.236475944519043, 0.810308575630188, 0.5877105593681335, 0.2672974765300751, 0.26597678661346436, 0.25135353207588196, 0.24978920817375183]
d loss, g loss: 0.18357145227491856 [9.29763412475586, 0.6446003913879395, 0.6374925374984741, 0.2651514410972595, 0

d loss, g loss: 0.08115506358444691 [6.406962871551514, 0.8246130347251892, 0.8858494758605957, 0.1540893018245697, 0.16667410731315613, 0.13890615105628967, 0.15886704623699188]
d loss, g loss: 0.09052800387144089 [6.159511566162109, 0.8943986892700195, 0.8903542757034302, 0.1406978964805603, 0.15657928586006165, 0.13763532042503357, 0.14276202023029327]
d loss, g loss: 0.06150531955063343 [6.401431083679199, 0.9436166882514954, 0.976540207862854, 0.14901547133922577, 0.15853941440582275, 0.12971259653568268, 0.15143239498138428]
d loss, g loss: 0.08167448081076145 [6.665950775146484, 0.9118765592575073, 0.9159289598464966, 0.16728954017162323, 0.16360758244991302, 0.15330851078033447, 0.15252622961997986]
d loss, g loss: 0.07158894557505846 [6.653622627258301, 0.8472015261650085, 0.9669536352157593, 0.16082078218460083, 0.16932465136051178, 0.14910109341144562, 0.15850166976451874]
d loss, g loss: 0.06991503573954105 [6.483159065246582, 0.9025189280509949, 0.939153790473938, 0.149802

d loss, g loss: 0.055842364323325455 [7.264450550079346, 1.089060664176941, 1.7725410461425781, 0.1466781497001648, 0.1621248871088028, 0.13232606649398804, 0.13063761591911316]
d loss, g loss: 0.07423786912113428 [6.02878475189209, 0.9297215342521667, 0.9706661105155945, 0.13956815004348755, 0.14184172451496124, 0.13215777277946472, 0.13070189952850342]
d loss, g loss: 0.07201817911118269 [6.180890083312988, 0.977715015411377, 1.085282325744629, 0.14332619309425354, 0.14210006594657898, 0.13138975203037262, 0.12133623659610748]


学習後のモデル（`G_AB, G_BA`）にテスト画像を入れて、うまく変換できているかを確認してみる。画素値が`[-1, 1]`に規格化されていることに注意。

In [12]:
import cv2

iii = cv2.imread("./apple2orange/testA/img/n07740461_10011.jpg")
# A_datagen(iii)
G_AB.predict(iii)

ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1462 predict_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1445 run_step  **
        outputs = mo

In [ ]:
cycle_gan.history

In [ ]:
!ls ./apple2orange/testA/img/

In [ ]:
!ls "./apple2orange/testA/img/n07740461_10011.jpg"

In [ ]:

# Model
results = G_AB.predict(next(A_generator))

In [ ]:
# cv2.imshow("img",np.array(results[0]*127+127).astype(int))
# plt.imshow(np.array(results[0]*127+127).astype(int))

In [ ]:
# cv2.imshow()

In [ ]:
type(Model)